In [51]:
from FakeModel import Model


import numpy as np
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper

In [52]:
editor = Editor();

## Example formatting for FakeModel

In [53]:
model_instance = Model("abc")
res = model_instance()

print(res)

{'PE': 'Unacceptable', 'KE': 'Unacceptable', 'LCE': 'Unacceptable'}


## Valid Definitions

In [123]:
PE_ACCEPTABLE = [
    "energy at rest",
    "energy that a body has because of its position relative to other bodies",
    "energy stored in the car at the top of the initial drop",
]
PE_UNACCEPTABLE = [
    "energy in motion",
    "energy lost as the car goes down the hill",
    "opposite of kinetic energy",
]
PE_INSUFFICIENT = [
    "4 Joules at the top of the roller coaster",
    "measured in Joules",
    "equal to m*h*9.8",
]

KE_ACCEPTABLE = [
    "energy in motion",
    "energy that the car has because it is moving",
    "the work needed to accelerate the roller coaster car from rest",
]
KE_UNACCEPTABLE = [
    "energy at rest",
    "never lost nor gained as the car moves",
    "the opposite of potantial energy",
]
KE_INSUFFICIENT = [
    "4 Joules at the bottom of the roller coaster",
    "measured in Joules",
    "equal to .5*m*v^2",
]

LCE_ACCEPTABLE = [
    "energy cannot be created or destroyed, only transformed",
    "the total energy of an isolated system remains constant",
    "energy can be converted from one form to another, but never created or destroyed",
]
LCE_UNACCEPTABLE = [
    "energy can be created or destroyed, not transformed",
    "the total energy of an isolated system will change",
]
LCE_INSUFFICIENT = [
    "K1 + U1 = K2 + U2",
    "potential energy transforms into kinetic energy",
    "As the car goes down the hill, some energy is lost to friction as heat",
]


## Function for getting data and frequencies

In [142]:
def getData(samples):
    data = {
        'PE':[],
        'KE':[],
        'LCE':[],
    }
    frequencies = {
        'PE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
        'KE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
        'LCE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
    }
    for sample in samples.data:
        d = Model(sample)()
        data['PE'].append(d['PE'])
        data['KE'].append(d['KE'])
        data['LCE'].append(d['LCE'])
        frequencies['PE'][d['PE']] += 1
        frequencies['KE'][d['KE']] += 1
        frequencies['LCE'][d['LCE']] += 1
    
    return data, frequencies


# **Vocab + POS**

## Minimum Functionality Tests
### Check for correct definitions
Should return acceptable

In [147]:
samples = editor.template(
    "Potential energy is {pe_acceptable}. Kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_acceptable}.", 
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE, nsamples=10, labels=0
)

data, frequencies = getData(samples)


### Check for incorrect definitions
should return unacceptable

In [148]:
samples = editor.template(
    "Potential energy is {pe_unacceptable}. Kinetic energy is {ke_unacceptable}. The Law of Conservation of energy states {lce_unacceptable}.",
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE, nsamples=10, labels=0
)
samples += editor.template(
    "Potential energy is not {pe_acceptable}. Kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_acceptable}.",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE, nsamples=10, labels=0
)

data, freqs = getData(samples)


### Check insufficient definitions
should return insufficient

In [126]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. Kinetic energy is {ke_insufficient}. The Law of Conservation of energy states {lce_insufficient}.",
    pe_insufficient=PE_INSUFFICIENT, ke_insufficient=KE_INSUFFICIENT, lce_insufficient=LCE_INSUFFICIENT, nsamples=10, labels=0
)

data, freqs = getData(samples)

{'PE': ['Acceptable', 'Unacceptable', 'Acceptable', 'Acceptable', 'Insufficient', 'Insufficient', 'Unacceptable', 'Unacceptable', 'Acceptable', 'Insufficient'], 'KE': ['Unacceptable', 'Acceptable', 'Acceptable', 'Insufficient', 'Insufficient', 'Unacceptable', 'Acceptable', 'Unacceptable', 'Insufficient', 'Unacceptable'], 'LCE': ['Unacceptable', 'Insufficient', 'Insufficient', 'Acceptable', 'Acceptable', 'Acceptable', 'Acceptable', 'Unacceptable', 'Acceptable', 'Acceptable']}


## Directional Tests

### Correct definitions followed by incorrect definition
correctness should not go up

In [149]:
samples = editor.template(
    "Potential energy is {pe_acceptable}. potential energy is {pe_unacceptable}. Kinetic energy is {ke_acceptable}. kinetic energy is {ke_unacceptable}. The Law of Conservation of energy states {lce_acceptable}."
    "The Law of Conservation of energy states {lce_unacceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    nsamples=50, labels=0
)

data, freqs = getData(samples)

### Incorrect Definition followed by correct definition
correctness should not go down

In [28]:
# code
samples = editor.template(
    "Potential energy is {pe_unacceptable}. potential energy is {pe_acceptable}. Kinetic energy is {ke_unacceptable}. kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_unacceptable}."
    "The Law of Conservation of energy states {lce_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    nsamples=50, labels=0
)


data, freqs = getData(samples)

### Insufficient definition followed by correct definition
Correctness should not go down

In [151]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. potential energy is {pe_acceptable}. Kinetic energy is {ke_insufficient}. kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_insufficient}."
    "The Law of Conservation of energy states {lce_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_insufficient=PE_INSUFFICIENT, ke_insufficient=KE_INSUFFICIENT, lce_insufficient=LCE_INSUFFICIENT,
    nsamples=50, labels=0
)

data, freqs = getData(samples)

### Insufficient definition followed by incorrect definition
Correctness should not go up

In [30]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. potential energy is {pe_unacceptable}. Kinetic energy is {ke_insufficient}. kinetic energy is {ke_unacceptable}. The Law of Conservation of energy states {lce_insufficient}."
    "The Law of Conservation of energy states {lce_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    pe_insufficient=PE_INSUFFICIENT, ke_insufficient=KE_INSUFFICIENT, lce_insufficient=LCE_INSUFFICIENT,
    nsamples=50, labels=0
)

data, freqs = getData(samples)


# Negation

## MFTs

In [31]:
# code

# Robustness

## INV

In [32]:
# code

# Taxonomy

## INV

In [33]:
# code

# Fairness

## INV

In [34]:
# code